In [1]:
%matplotlib widget

import math, numba, pprint, pylab
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

from itertools import islice
from ipywidgets import (interact, interact_manual, fixed, FloatSlider, Dropdown)

pylab.rcParams['figure.figsize'] = (12.0, 6.0)
plt.style.use('dark_background')

In [2]:
def tg_pprint(*args, width=200, **kwargs):
    return pprint.pprint(*args, **kwargs, width=width)

In [3]:
@numba.jit
def log_map_gen(x0=.5, r=2.4):
    xn1=x0
    yield xn1
    while True:
        xn=xn1
        xn1 = r*xn*(1.-xn)
        yield xn1

In [4]:
%matplotlib widget
@interact(   x0=FloatSlider(.5, min=0., max=1., step=.001)
            ,r= FloatSlider(2.98, min=0., max=4., step=.001)
#            ,n=[('30', 30), ('100', 100), ('200', 200)] # n=fixed(50)
            ,n=Dropdown(options=[30, 100, 200])
         )
def graph_pop(x0=.5, r=2.4, n=30):
    ts = range(n)
    xs = tuple(islice(log_map_gen(x0=x0, r=r), n))
    #tg_pprint(xs)
    plt.gcf().clear()
    plt.ylim((0, 1.))
    plt.plot(ts, xs, marker=',')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
@numba.jit
def gen_rs(rmin=1., rmax=4., nbins=2000):
    delta = rmax - rmin
    for i in range(nbins):
        yield rmin + delta * i / nbins

In [15]:
def foobar(rmin_=0, rmax_=4.0):
    @interact_manual(    rmin=FloatSlider(rmin_, min=rmin_, max=rmax_, step=0.0001)
                        ,rmax=FloatSlider(rmax_, min=rmin_, max=rmax_, step=0.0001)
                        ,nthrow=Dropdown(options=(0, 10, 20, 40, 80, 160, 320, 640), index=4) # nthrow=fixed(50)
                        ,nkeep= Dropdown(options=(   10, 20, 40, 80, 160, 320, 640), index=3)
                        ,nbins= Dropdown(options=(8000, 16000, 32000, 64000, 128000, 256000, 512000), index=4)
                    )
    def plot_bifurication_map(rmin=0, rmax=4, nthrow=50, nkeep=50, nbins=2000):
        #plt.yscale('linear')
        rs = []
        xs = []
        for r in gen_rs(rmin=rmin, rmax=rmax, nbins=nbins):
            x = tuple(islice(log_map_gen(r=r), nthrow, nthrow+nkeep))
            rs.append(len(x)*(r,))
            xs.append(x)
        plt.gcf().clear()
        plt.plot(rs, xs, marker='.', linewidth=0, markersize=.2)

In [16]:
%matplotlib widget
foobar(0, 4)

interactive(children=(FloatSlider(value=0.0, description='rmin', max=4.0, step=0.0001), FloatSlider(value=4.0,…

In [17]:
#import bokeh.plotting as plotting

#plotting.output_notebook()

#f = plotting.figure(plot_width=600, plot_height=400)
#f.circle(*zip(*gen_r_e(nbins=500000)),size=.5)
#plotting.show(f)

In [9]:
def gen_mandel(xrng=(-2,.5), yrng=(-1,1), horizon=2, nbins=100, niter=32):
    rng = ((xrng[1] - xrng[0])
          ,(yrng[1] - yrng[0])
          )
    delta = tuple(map(lambda v: v/nbins, rng))

    xs = np.linspace(*xrng, nbins, dtype=float)
    ys = np.linspace(*yrng, nbins, dtype=float)
    cs = xs + ys[:, None]*1j
    ns = np.zeros(cs.shape, dtype=int)
    zs = np.zeros(cs.shape, dtype=np.complex64)
    m  = np.zeros((nbins, nbins), dtype=complex)
    
    for it in range(niter):
        i = np.less(abs(zs), horizon)
        ns[i] = it
        zs[i] = zs[i]**2 + cs[i]
    ns[ns == niter - 1] = 0
    
    log_horizon = np.log(np.log(horizon))/np.log(2)
    with np.errstate(invalid='ignore'):
        zs = np.nan_to_num(ns + 1 - np.log(np.log(abs(zs))/np.log(2) + log_horizon))
    return zs, ns, cs

In [10]:
zs, ns, cs = gen_mandel(nbins=1000, horizon=2**10)
#plt.countour(xs, ys, np.absolute(m))
plt.imshow(np.absolute(zs))
#tg_pprint(zs)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
from mpl_toolkits import mplot3d

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.view_init(azim=-90, elev=90)
ax.scatter3D(cs.real, cs.imag, abs(zs), c=abs(zs).flat, marker='.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
abs(2+-1.324j)

2.398536220281028

In [13]:
print(np.finfo(np.float64))

Machine parameters for float64
---------------------------------------------------------------
precision =  15   resolution = 1.0000000000000001e-15
machep =    -52   eps =        2.2204460492503131e-16
negep =     -53   epsneg =     1.1102230246251565e-16
minexp =  -1022   tiny =       2.2250738585072014e-308
maxexp =   1024   max =        1.7976931348623157e+308
nexp =       11   min =        -max
---------------------------------------------------------------



In [14]:
import ipywidgets as widgets